In [ ]:
# Add this cell at the beginning of your Kaggle notebook
!pip install face-recognition
!pip install mtcnn
!pip install keras-facenet
import cv2
import numpy as np
import os
import pickle
import json
from datetime import datetime
from pathlib import Path
import argparse
from PIL import Image, ImageEnhance
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
import cv2
    
def show_image(image, title="Image"):
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10, 8))
        plt.imshow(rgb_image)
        plt.title(title)
        plt.axis('off')
        plt.show()

# Try to import face_recognition library (most reliable)
try:
    import face_recognition
    FACE_RECOGNITION_AVAILABLE = True
    print("✓ Using face_recognition library (recommended)")
except ImportError:
    FACE_RECOGNITION_AVAILABLE = False
    print("⚠ face_recognition not available. Install with: pip install face-recognition")

# Try to import MTCNN for face detection
try:
    from mtcnn import MTCNN
    MTCNN_AVAILABLE = True
    print("✓ MTCNN face detector available")
except ImportError:
    MTCNN_AVAILABLE = False
    print("⚠ MTCNN not available. Install with: pip install mtcnn tensorflow")

# Try to import FaceNet model
try:
    from keras_facenet import FaceNet
    FACENET_AVAILABLE = True
    print("✓ FaceNet model available")
except ImportError:
    FACENET_AVAILABLE = False
    print("⚠ FaceNet not available. Install with: pip install keras-facenet")

# Deep learning imports
try:
    import torch
    import torch.nn as nn
    import torchvision.transforms as transforms
    from sklearn.metrics.pairwise import cosine_similarity
    TORCH_AVAILABLE = True
except ImportError:
    TORCH_AVAILABLE = False
    print("⚠ PyTorch not available for enhanced features")


class ProfessionalFaceRecognitionSystem:
    """Professional Face Recognition System using state-of-the-art models"""
    
    def __init__(self, known_faces_dir="/kaggle/working/face-recognition-dataset/Original Images/Original Images", encodings_file="/kaggle/working/face-encoding/face_encodings.pkl"):
        if known_faces_dir is None:
            # Check what's actually in your Kaggle input
            input_dirs = os.listdir("/kaggle/input")
            print(f"Available input directories: {input_dirs}")
            
            # Try to find the face recognition dataset
            for dirname in input_dirs:
                full_path = f"/kaggle/input/{dirname}"
                if os.path.isdir(full_path):
                    # Check if it contains subdirectories (person folders)
                    subdirs = [d for d in os.listdir(full_path) if os.path.isdir(os.path.join(full_path, d))]
                    if len(subdirs) > 0:
                        known_faces_dir = full_path
                        print(f"Found dataset at: {known_faces_dir}")
                        print(f"Contains folders: {subdirs}")
                        break
            
            if known_faces_dir is None:
                known_faces_dir = "/kaggle/input"
                print("Using default input directory")
        self.known_faces_dir = Path(known_faces_dir)
        self.encodings_file = encodings_file
        self.known_face_encodings = []
        self.known_face_names = []
        
        # Create directories
        self.known_faces_dir.mkdir(exist_ok=True)
        
        # Initialize face detection and recognition models
        self.init_models()
        
        # Load or create encodings
        self.load_or_create_encodings()
    
    def init_models(self):
        """Initialize the best available face detection and recognition models"""
        print("Initializing face detection and recognition models...")
        
        # Method 1: face_recognition library (most reliable)
        if FACE_RECOGNITION_AVAILABLE:
            self.detection_method = "face_recognition"
            self.encoding_method = "face_recognition"
            print("✓ Using face_recognition library for detection and encoding")
            return
        
        # Method 2: MTCNN + FaceNet
        if MTCNN_AVAILABLE and FACENET_AVAILABLE:
            self.detector = MTCNN()
            self.embedder = FaceNet()
            self.detection_method = "mtcnn"
            self.encoding_method = "facenet"
            print("✓ Using MTCNN + FaceNet combination")
            return
        
        # Method 3: MTCNN + Custom embeddings
        if MTCNN_AVAILABLE:
            self.detector = MTCNN()
            self.detection_method = "mtcnn"
            self.encoding_method = "custom"
            print("✓ Using MTCNN detector with custom embeddings")
            return
        
        # Fallback: OpenCV DNN face detector
        self.init_opencv_dnn()
        self.detection_method = "opencv_dnn"
        self.encoding_method = "custom"
    
    def init_opencv_dnn(self):
        """Initialize OpenCV DNN face detector"""
        try:
            # Download models if needed
            self.download_opencv_models()
            
            # Load the DNN model
            self.net = cv2.dnn.readNetFromTensorflow(
                'models/opencv_face_detector_uint8.pb',
                'models/opencv_face_detector.pbtxt'
            )
            print("✓ OpenCV DNN face detector loaded")
        except Exception as e:
            print(f"Failed to load OpenCV DNN: {e}")
            # Ultimate fallback to Haar Cascade
            self.face_cascade = cv2.CascadeClassifier(
                cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
            )
            self.detection_method = "haar"
            print("✓ Using Haar Cascade as final fallback")
    
    def download_opencv_models(self):
        """Download OpenCV face detection models"""
        import urllib.request
        
        models_dir = Path("models")
        models_dir.mkdir(exist_ok=True)
        
        models = {
            'opencv_face_detector_uint8.pb': 'https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20170830/opencv_face_detector_uint8.pb',
            'opencv_face_detector.pbtxt': 'https://raw.githubusercontent.com/opencv/opencv_3rdparty/dnn_samples_face_detector_20170830/opencv_face_detector.pbtxt'
        }
        
        for filename, url in models.items():
            filepath = models_dir / filename
            if not filepath.exists():
                print(f"Downloading {filename}...")
                try:
                    urllib.request.urlretrieve(url, filepath)
                    print(f"✓ Downloaded {filename}")
                except Exception as e:
                    print(f"Failed to download {filename}: {e}")
    
    def detect_faces(self, image):
        """Detect faces using the best available method"""
        if self.detection_method == "face_recognition":
            return self.detect_faces_face_recognition(image)
        elif self.detection_method == "mtcnn":
            return self.detect_faces_mtcnn(image)
        elif self.detection_method == "opencv_dnn":
            return self.detect_faces_opencv_dnn(image)
        else:
            return self.detect_faces_haar(image)
    
    def detect_faces_face_recognition(self, image):
        """Detect faces using face_recognition library"""
        # Convert BGR to RGB
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_image, model="hog")
        
        # Convert to our format (x, y, w, h)
        faces = []
        for (top, right, bottom, left) in face_locations:
            faces.append((left, top, right - left, bottom - top, 1.0))
        
        return faces
    
    def detect_faces_mtcnn(self, image):
        """Detect faces using MTCNN"""
        # Convert BGR to RGB
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        result = self.detector.detect_faces(rgb_image)
        
        faces = []
        for face in result:
            if face['confidence'] > 0.9:  # High confidence threshold
                x, y, w, h = face['box']
                faces.append((x, y, w, h, face['confidence']))
        
        return faces
    
    def detect_faces_opencv_dnn(self, image):
        """Detect faces using OpenCV DNN"""
        h, w = image.shape[:2]
        blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300), [104, 117, 123])
        self.net.setInput(blob)
        detections = self.net.forward()
        
        faces = []
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.7:  # Higher confidence threshold
                x1 = int(detections[0, 0, i, 3] * w)
                y1 = int(detections[0, 0, i, 4] * h)
                x2 = int(detections[0, 0, i, 5] * w)
                y2 = int(detections[0, 0, i, 6] * h)
                faces.append((x1, y1, x2 - x1, y2 - y1, confidence))
        
        return faces
    
    def detect_faces_haar(self, image):
        """Detect faces using Haar Cascade"""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(
            gray, scaleFactor=1.1, minNeighbors=5, minSize=(50, 50)
        )
        return [(x, y, w, h, 1.0) for (x, y, w, h) in faces]
    
    def extract_face_encoding(self, image, face_location=None):
        """Extract face encoding using the best available method"""
        if self.encoding_method == "face_recognition":
            return self.extract_encoding_face_recognition(image, face_location)
        elif self.encoding_method == "facenet":
            return self.extract_encoding_facenet(image, face_location)
        else:
            return self.extract_encoding_custom(image, face_location)
    
    def extract_encoding_face_recognition(self, image, face_location):
        """Extract face encoding using face_recognition library"""
        # Convert BGR to RGB
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if face_location:
            x, y, w, h = face_location[:4]
            # Convert to face_recognition format (top, right, bottom, left)
            face_locations = [(y, x + w, y + h, x)]
        else:
            face_locations = face_recognition.face_locations(rgb_image)
        
        if not face_locations:
            return None
        
        # Get the encoding for the first face
        encodings = face_recognition.face_encodings(rgb_image, face_locations)
        if encodings:
            return encodings[0]
        
        return None
    
    def extract_encoding_facenet(self, image, face_location):
        """Extract face encoding using FaceNet"""
        if face_location:
            x, y, w, h = face_location[:4]
            face_image = image[y:y+h, x:x+w]
        else:
            face_image = image
        
        # Preprocess for FaceNet
        face_image = cv2.cvtColor(face_image, cv2.COLOR_BGR2RGB)
        face_image = cv2.resize(face_image, (160, 160))
        face_image = np.expand_dims(face_image, axis=0)
        face_image = face_image.astype('float32') / 255.0
        
        try:
            embedding = self.embedder.embeddings(face_image)
            return embedding[0]
        except Exception as e:
            print(f"FaceNet encoding error: {e}")
            return None
    
    def extract_encoding_custom(self, image, face_location):
        """Extract custom face encoding using traditional methods"""
        if face_location:
            x, y, w, h = face_location[:4]
            face_image = image[y:y+h, x:x+w]
        else:
            face_image = image
        
        # Enhanced preprocessing
        gray = cv2.cvtColor(face_image, cv2.COLOR_BGR2GRAY)
        
        # Normalize lighting
        clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
        enhanced = clahe.apply(gray)
        
        # Resize to standard size
        resized = cv2.resize(enhanced, (128, 128))
        
        # Extract multiple features
        # 1. Raw pixel values
        pixel_features = resized.flatten()
        
        # 2. LBP features
        lbp_features = self.extract_lbp_features(resized)
        
        # 3. Gradient features
        grad_x = cv2.Sobel(resized, cv2.CV_64F, 1, 0, ksize=3)
        grad_y = cv2.Sobel(resized, cv2.CV_64F, 0, 1, ksize=3)
        gradient_features = np.concatenate([grad_x.flatten(), grad_y.flatten()])
        
        # Combine all features
        combined_features = np.concatenate([
            pixel_features * 0.5,  # Reduced weight for raw pixels
            lbp_features * 2.0,    # Higher weight for LBP
            gradient_features * 0.3  # Lower weight for gradients
        ])
        
        # Normalize
        norm = np.linalg.norm(combined_features)
        if norm > 0:
            combined_features = combined_features / norm
        
        return combined_features.astype(np.float32)
    
    def extract_lbp_features(self, image):
        """Extract Local Binary Pattern features"""
        # Simple LBP implementation
        rows, cols = image.shape
        lbp = np.zeros((rows, cols), dtype=np.uint8)
        
        for i in range(1, rows - 1):
            for j in range(1, cols - 1):
                center = image[i, j]
                code = 0
                
                # 8-neighborhood
                neighbors = [
                    image[i-1, j-1], image[i-1, j], image[i-1, j+1],
                    image[i, j+1], image[i+1, j+1], image[i+1, j],
                    image[i+1, j-1], image[i, j-1]
                ]
                
                for k, neighbor in enumerate(neighbors):
                    if neighbor >= center:
                        code |= (1 << k)
                
                lbp[i, j] = code
        
        # Create histogram
        hist, _ = np.histogram(lbp.ravel(), bins=256, range=(0, 256))
        return hist.astype(np.float32)
    
    def load_or_create_encodings(self):
        """Load existing encodings or create new ones"""
        if os.path.exists(self.encodings_file):
            print("Loading existing face encodings...")
            try:
                with open(self.encodings_file, 'rb') as f:
                    data = pickle.load(f)
                    self.known_face_encodings = data.get('encodings', [])
                    self.known_face_names = data.get('names', [])
                
                unique_people = len(set(self.known_face_names))
                print(f"✓ Loaded encodings for {unique_people} people ({len(self.known_face_encodings)} total)")
                
                if unique_people == 0:
                    print("No valid encodings found, creating new ones...")
                    self.create_face_encodings()
                    
            except Exception as e:
                print(f"Error loading encodings: {e}")
                self.create_face_encodings()
        else:
            print("Creating new face encodings...")
            self.create_face_encodings()
    
    def create_face_encodings(self):
        """Create face encodings from known faces directory"""
        print(f"Scanning for known faces in {self.known_faces_dir}...")
        
        image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']
        encodings = []
        names = []
        
        if not any(self.known_faces_dir.iterdir()):
            print(f"⚠ No directories found in {self.known_faces_dir}")
            print("Please create subdirectories with person names and place their images inside.")
            return
        
        for person_dir in self.known_faces_dir.iterdir():
            if person_dir.is_dir():
                person_name = person_dir.name
                print(f"\nProcessing {person_name}...")
                
                person_images = list(person_dir.glob('*'))
                person_images = [img for img in person_images if img.suffix.lower() in image_extensions]
                
                if not person_images:
                    print(f"  ⚠ No valid images found for {person_name}")
                    continue
                
                person_encodings = []
                successful_images = 0
                
                for img_path in person_images:
                    try:
                        # Load image
                        image = cv2.imread(str(img_path))
                        if image is None:
                            print(f"  ✗ Could not load {img_path.name}")
                            continue
                        
                        # Detect faces
                        faces = self.detect_faces(image)
                        
                        if not faces:
                            print(f"  ✗ No face detected in {img_path.name}")
                            continue
                        
                        if len(faces) > 1:
                            print(f"  ⚠ Multiple faces in {img_path.name}, using largest")
                            faces = [max(faces, key=lambda x: x[2] * x[3])]  # Largest face
                        
                        # Extract encoding
                        face_location = faces[0]
                        encoding = self.extract_face_encoding(image, face_location)
                        
                        if encoding is not None:
                            person_encodings.append(encoding)
                            successful_images += 1
                            print(f"  ✓ Processed {img_path.name}")
                        else:
                            print(f"  ✗ Could not extract encoding from {img_path.name}")
                            
                    except Exception as e:
                        print(f"  ✗ Error processing {img_path.name}: {e}")
                
                # Add encodings for this person
                if person_encodings:
                    encodings.extend(person_encodings)
                    names.extend([person_name] * len(person_encodings))
                    print(f"  → Successfully processed {successful_images}/{len(person_images)} images")
                else:
                    print(f"  ✗ No valid encodings created for {person_name}")
        
        self.known_face_encodings = encodings
        self.known_face_names = names
        
        if encodings:
            self.save_encodings()
            unique_people = len(set(names))
            print(f"\n✓ Database created: {unique_people} people, {len(encodings)} total encodings")
        else:
            print("\n⚠ No face encodings were created!")
            print("Please check that:")
            print("1. Images are in person-name subdirectories")
            print("2. Images contain clear, front-facing faces")
            print("3. Image files are in supported formats (jpg, png, etc.)")
    
    def save_encodings(self):
        """Save encodings to file"""
        data = {
            'encodings': self.known_face_encodings,
            'names': self.known_face_names,
            'metadata': {
                'created': datetime.now().isoformat(),
                'detection_method': self.detection_method,
                'encoding_method': self.encoding_method,
                'total_people': len(set(self.known_face_names)),
                'total_encodings': len(self.known_face_encodings)
            }
        }
        
        with open(self.encodings_file, 'wb') as f:
            pickle.dump(data, f)
        
        print(f"✓ Encodings saved to {self.encodings_file}")
    
    def recognize_face(self, face_encoding, threshold=None):
        """Recognize a face from its encoding"""
        if not self.known_face_encodings or face_encoding is None:
            return "Unknown", 0.0
        
        # Set threshold based on encoding method
        if threshold is None:
            if self.encoding_method == "face_recognition":
                threshold = 0.6  # face_recognition uses distance, lower is better
            else:
                threshold = 0.7  # Custom encodings use similarity, higher is better
        
        if self.encoding_method == "face_recognition":
            # Use face_recognition's built-in comparison
            distances = face_recognition.face_distance(self.known_face_encodings, face_encoding)
            
            # Group by person and find best match
            person_distances = {}
            for i, distance in enumerate(distances):
                name = self.known_face_names[i]
                if name not in person_distances:
                    person_distances[name] = []
                person_distances[name].append(distance)
            
            # Find person with minimum average distance
            best_person = None
            best_distance = float('inf')
            
            for name, dist_list in person_distances.items():
                avg_distance = np.mean(dist_list)
                if avg_distance < best_distance:
                    best_distance = avg_distance
                    best_person = name
            
            # Convert distance to similarity for consistent output
            similarity = max(0, 1.0 - best_distance)
            
            if best_distance <= threshold:
                return best_person, similarity
            else:
                return "Unknown", similarity
                
        else:
            # Use cosine similarity for custom encodings
            similarities = cosine_similarity([face_encoding], self.known_face_encodings)[0]
            
            # Group by person
            person_similarities = {}
            for i, sim in enumerate(similarities):
                name = self.known_face_names[i]
                if name not in person_similarities:
                    person_similarities[name] = []
                person_similarities[name].append(sim)
            
            # Find person with highest average similarity
            best_person = None
            best_similarity = -1
            
            for name, sim_list in person_similarities.items():
                avg_similarity = np.mean(sim_list)
                if avg_similarity > best_similarity:
                    best_similarity = avg_similarity
                    best_person = name
            
            if best_similarity >= threshold:
                return best_person, best_similarity
            else:
                return "Unknown", best_similarity
    

    
    def recognize_faces_in_image(self, image_path, threshold=None):
        """Recognize all faces in an image and display with bounding boxes and name tags (matplotlib, no save)."""
        try:
            image = cv2.imread(str(image_path))
            if image is None:
                print(f"Could not load image: {image_path}")
                return []
    
            # Detect faces
            faces = self.detect_faces(image)
            if not faces:
                print("No faces detected in the image")
                return []
    
            results = []
            print(f"Found {len(faces)} face(s), processing...")
    
            for i, face_data in enumerate(faces):
                x, y, w, h = face_data[:4]
                confidence = face_data[4] if len(face_data) > 4 else 1.0
    
                # Skip very small faces
                if w < 40 or h < 40:
                    print(f"Skipping small face {i+1}: {w}x{h}")
                    continue
    
                # Extract face encoding
                encoding = self.extract_face_encoding(image, (x, y, w, h))
    
                if encoding is not None:
                    # Recognize face
                    name, similarity = self.recognize_face(encoding, threshold)
    
                    results.append({
                        'face_id': i + 1,
                        'name': name,
                        'confidence': similarity,
                        'detection_confidence': confidence,
                        'location': (x, y, x + w, y + h),
                        'size': (w, h)
                    })
    
                    print(f"Face {i+1}: {name} (confidence: {similarity:.3f})")
    
                    # Draw bounding box
                    color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
                    cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    
                    # Prepare label
                    label = f"{name} ({similarity:.2f})" if name != "Unknown" else f"Unknown ({similarity:.2f})"
                    label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                    label_y = y - 10 if y - 10 > 10 else y + 10
                    cv2.rectangle(image, (x, label_y - label_size[1] - 5), (x + label_size[0], label_y + 5), color, cv2.FILLED)
                    cv2.putText(image, label, (x, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
                else:
                    print(f"Could not extract encoding for face {i+1}")
    
            # Show the image with bounding boxes and labels using matplotlib
            show_image(image, title="Recognized Faces")
    
            return results
    
        except Exception as e:
            print(f"Error processing image: {e}")
            return []


    def recognize_faces_in_video_notebook(self, video_source=0, threshold=None, max_frames=100):
            cap = cv2.VideoCapture(video_source)
            if not cap.isOpened():
                print(f"Error: Could not open video source {video_source}")
                return
        
            frame_count = 0
            while frame_count < max_frames:
                ret, frame = cap.read()
                if not ret:
                    break
        
                faces = self.detect_faces(frame)
                for i, face_data in enumerate(faces):
                    x, y, w, h = face_data[:4]
                    confidence = face_data[4] if len(face_data) > 4 else 1.0
                    if w < 40 or h < 40:
                        continue
                    encoding = self.extract_face_encoding(frame, (x, y, w, h))
                    if encoding is not None:
                        name, similarity = self.recognize_face(encoding, threshold)
                        color = (0, 255, 0) if name != "Unknown" else (0, 0, 255)
                        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
                        label = f"{name} ({similarity:.2f})"
                        label_size, _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
                        label_y = y - 10 if y - 10 > 10 else y + 10
                        cv2.rectangle(frame, (x, label_y - label_size[1] - 5), (x + label_size[0], label_y + 5), color, cv2.FILLED)
                        cv2.putText(frame, label, (x, label_y), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
                # Display frame using matplotlib
                rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                plt.figure(figsize=(10, 8))
                plt.imshow(rgb_frame)
                plt.axis('off')
                plt.title(f"Frame {frame_count+1}")
                plt.show()
        
                frame_count += 1
                # Optional: break early for demo
                # if frame_count > 10: break
        
            cap.release()

        
    
    def add_person(self, name, image_path):
        """Add a new person from an image"""
        try:
            image = cv2.imread(str(image_path))
            if image is None:
                print(f"Could not load image: {image_path}")
                return False
            
            faces = self.detect_faces(image)
            
            if not faces:
                print("No face detected in the image")
                return False
            
            if len(faces) > 1:
                print("Multiple faces detected, using the largest one")
                faces = [max(faces, key=lambda x: x[2] * x[3])]
            
            # Extract encoding
            face_location = faces[0]
            encoding = self.extract_face_encoding(image, face_location)
            
            if encoding is None:
                print("Could not extract face encoding")
                return False
            
            # Add to database
            self.known_face_encodings.append(encoding)
            self.known_face_names.append(name)
            
            # Save updated database
            self.save_encodings()
            
            print(f"✓ Added {name} to the database")
            return True
            
        except Exception as e:
            print(f"Error adding person: {e}")
            return False
    
    def remove_person(self, name):
        """Remove all encodings for a person"""
        indices_to_remove = [i for i, n in enumerate(self.known_face_names) if n == name]
        
        if not indices_to_remove:
            print(f"Person '{name}' not found in database")
            return False
        
        # Remove in reverse order to maintain indices
        for i in reversed(indices_to_remove):
            del self.known_face_encodings[i]
            del self.known_face_names[i]
        
        self.save_encodings()
        print(f"✓ Removed {len(indices_to_remove)} encodings for '{name}'")
        return True
    
    def list_people(self):
        """List all people in the database"""
        if not self.known_face_names:
            print("No people in the database")
            return
        
        # Count encodings per person
        person_counts = {}
        for name in self.known_face_names:
            person_counts[name] = person_counts.get(name, 0) + 1
        
        print("\nPeople in Database:")
        print("-" * 40)
        for name, count in sorted(person_counts.items()):
            print(f"  {name}: {count} encoding(s)")
        
        print(f"\nTotal: {len(person_counts)} people, {len(self.known_face_names)} encodings")
    
    def get_system_info(self):
        """Get system information"""
        return {
            'detection_method': self.detection_method,
            'encoding_method': self.encoding_method,
            'known_people': len(set(self.known_face_names)),
            'total_encodings': len(self.known_face_encodings),
            'available_libraries': {
                'face_recognition': FACE_RECOGNITION_AVAILABLE,
                'mtcnn': MTCNN_AVAILABLE,
                'facenet': FACENET_AVAILABLE,
                'torch': TORCH_AVAILABLE
            }
        }
    
    def benchmark_recognition(self, test_image_path, iterations=10):
        """Benchmark recognition performance"""
        import time
        
        try:
            image = cv2.imread(str(test_image_path))
            if image is None:
                print(f"Could not load test image: {test_image_path}")
                return
            
            print(f"Benchmarking with {iterations} iterations...")
            
            # Face detection benchmark
            start_time = time.time()
            for _ in range(iterations):
                faces = self.detect_faces(image)
            detection_time = (time.time() - start_time) / iterations
            
            if not faces:
                print("No faces detected for benchmarking")
                return
            
            # Face encoding benchmark
            face_location = faces[0]
            start_time = time.time()
            for _ in range(iterations):
                encoding = self.extract_face_encoding(image, face_location)
            encoding_time = (time.time() - start_time) / iterations
            
            # Recognition benchmark
            if encoding is not None and self.known_face_encodings:
                start_time = time.time()
                for _ in range(iterations):
                    name, similarity = self.recognize_face(encoding)
                recognition_time = (time.time() - start_time) / iterations
            else:
                recognition_time = 0
            
            print(f"\nBenchmark Results:")
            print(f"  Face Detection: {detection_time*1000:.2f}ms per frame")
            print(f"  Face Encoding: {encoding_time*1000:.2f}ms per face")
            print(f"  Face Recognition: {recognition_time*1000:.2f}ms per face")
            print(f"  Total Pipeline: {(detection_time + encoding_time + recognition_time)*1000:.2f}ms per face")
            
        except Exception as e:
            print(f"Benchmark error: {e}")


# Main execution and CLI interface
def main():
    parser = argparse.ArgumentParser(description='Professional Face Recognition System')
    parser.add_argument('--mode', choices=['train', 'recognize', 'video', 'add', 'remove', 'list', 'info', 'benchmark'],
                       default='recognize', help='Operation mode')
    parser.add_argument('--image', type=str, help='Path to image file')
    parser.add_argument('--video', type=str, help='Path to video file or camera index (0, 1, etc.)')
    parser.add_argument('--output', type=str, help='Output video file path')
    parser.add_argument('--name', type=str, help='Person name for add/remove operations')
    parser.add_argument('--threshold', type=float, help='Recognition threshold')
    parser.add_argument('--known-faces', type=str, default='known_faces', 
                       help='Directory containing known faces')
    parser.add_argument('--encodings', type=str, default='face_encodings.pkl',
                       help='Face encodings file')
    
    args = parser.parse_args()
    
    # Initialize the system
    print("Initializing Professional Face Recognition System...")
    system = ProfessionalFaceRecognitionSystem(args.known_faces, args.encodings)
    
    if args.mode == 'train':
        print("Creating/updating face encodings database...")
        system.create_face_encodings()
        
    elif args.mode == 'recognize':
        if not args.image:
            print("Please specify an image file with --image")
            return
        
        print(f"Recognizing faces in: {args.image}")
        results = system.recognize_faces_in_image(args.image, args.threshold)
        
        if results:
            print(f"\nRecognition Results:")
            print("-" * 50)
            for result in results:
                print(f"Face {result['face_id']}: {result['name']} "
                      f"(confidence: {result['confidence']:.3f}, "
                      f"location: {result['location']}, "
                      f"size: {result['size']})")
        else:
            print("No faces recognized")
    
    elif args.mode == 'video':
        video_source = 0  # Default to webcam
        if args.video:
            if args.video.isdigit():
                video_source = int(args.video)
            else:
                video_source = args.video
        
        print(f"Starting video recognition from: {video_source}")
        system.recognize_faces_video(video_source, args.output, args.threshold)
    
    elif args.mode == 'add':
        if not args.name or not args.image:
            print("Please specify both --name and --image for adding a person")
            return
        
        success = system.add_person(args.name, args.image)
        if success:
            print(f"Successfully added {args.name}")
        else:
            print(f"Failed to add {args.name}")
    
    elif args.mode == 'remove':
        if not args.name:
            print("Please specify --name for removing a person")
            return
        
        success = system.remove_person(args.name)
        if success:
            print(f"Successfully removed {args.name}")
        else:
            print(f"Failed to remove {args.name}")
    
    elif args.mode == 'list':
        system.list_people()
    
    elif args.mode == 'info':
        info = system.get_system_info()
        print("\nSystem Information:")
        print("-" * 40)
        print(f"Detection Method: {info['detection_method']}")
        print(f"Encoding Method: {info['encoding_method']}")
        print(f"Known People: {info['known_people']}")
        print(f"Total Encodings: {info['total_encodings']}")
        print("\nAvailable Libraries:")
        for lib, available in info['available_libraries'].items():
            status = "✓" if available else "✗"
            print(f"  {status} {lib}")
    
    elif args.mode == 'benchmark':
        if not args.image:
            print("Please specify an image file with --image for benchmarking")
            return
        
        system.benchmark_recognition(args.image)


# Usage examples and documentation
def print_usage_examples():
    """Print usage examples"""
    examples = """
Usage Examples:
    
# Train the system (create face database)
python face_recognition_system.py --mode train

# Recognize faces in an image
python face_recognition_system.py --mode recognize --image path/to/image.jpg

# Real-time recognition from webcam
python face_recognition_system.py --mode video

# Recognize from video file
python face_recognition_system.py --mode video --video path/to/video.mp4

# Add a new person
python face_recognition_system.py --mode add --name "John Doe" --image path/to/john.jpg

# Remove a person
python face_recognition_system.py --mode remove --name "John Doe"

# List all known people
python face_recognition_system.py --mode list

# Show system information
python face_recognition_system.py --mode info

# Benchmark performance
python face_recognition_system.py --mode benchmark --image path/to/test.jpg

Setup Instructions:
1. Create a 'known_faces' directory
2. Create subdirectories for each person (e.g., 'known_faces/John_Doe/')
3. Place multiple photos of each person in their respective directories
4. Run training mode to create the face database
5. Use recognition modes to identify faces

Required Libraries (install as needed):
- pip install opencv-python numpy pillow
- pip install face-recognition (recommended)
- pip install mtcnn tensorflow (alternative)
- pip install keras-facenet (alternative)
- pip install torch torchvision (for enhanced features)
- pip install scikit-learn (for similarity calculations)
    """
    print(examples)


# For Kaggle notebook usage
if __name__ == "__main__":
    # Initialize the system
    print("Initializing Face Recognition System for Kaggle...")
    system = ProfessionalFaceRecognitionSystem()
    
    # List what we found
    print(f"Dataset directory: {system.known_faces_dir}")
    if system.known_faces_dir.exists():
        dirs = [d.name for d in system.known_faces_dir.iterdir() if d.is_dir()]
        print(f"Found person directories: {dirs}")
    
    # You can now run specific operations:
    # system.create_face_encodings()  # To train
    # system.list_people()  # To see who's in the database
    # results = system.recognize_faces_in_image("path/to/test/image.jpg")  # To recognize


In [ ]:
system.list_people()  # To see who's in the database


In [ ]:
results = system.recognize_faces_in_image("/kaggle/input/testimages/v1.jpg")  # To recognize


In [ ]:
results = system.recognize_faces_in_image("/kaggle/input/alexandradaddario/ad1.jpg")  # To recognize


In [ ]:
results = system.recognize_faces_in_image("/kaggle/input/henrycavill/hc1.webp")  # To recognize


In [ ]:
results = system.recognize_faces_in_image("/kaggle/input/henrycavill/hc2.jpg")  # To recognize


In [ ]:
results = system.recognize_faces_in_image("/kaggle/input/rock-zac/rock-zef.jpg")  # To recognize


In [ ]:
results = system.recognize_faces_in_image("/kaggle/input/zac-rock/a.webp")  # To recognize


In [ ]:
results = system.recognize_faces_in_image("/kaggle/input/viratk/121615492.avif")  # To recognize


In [ ]:
system.recognize_faces_in_video_notebook("/kaggle/input/vk12345/2025-06-13 18-01-00.mkv",max_frames=10)  # 0 for webcam, or video file path


In [ ]:
import shutil

# Source and destination
src = "/kaggle/input/face-recognition-dataset"
dst = "/kaggle/working/face-recognition-dataset"

# Copy entire folder
shutil.copytree(src, dst)
print("Dataset copied successfully!")


In [ ]:

import shutil

# Source and destination
src = "/kaggle/input/vanjikodi"
dst = "/kaggle/working/face-recognition-dataset/Original Images/Original Images/Vanjikodi"

# Copy entire folder
shutil.copytree(src, dst)
print("Dataset copied successfully!")


In [ ]:
import shutil

# Source and destination
src = "/kaggle/input/face-encodings"
dst = "/kaggle/working/face-encoding"

# Copy entire folder
shutil.copytree(src, dst)
print("Dataset copied successfully!")


In [ ]:
system.add_person("VanjiKodi", "/kaggle/working/face-recognition-dataset/Original Images/Original Images/Vanjikodi/WhatsApp Image 2025-06-13 at 6.12.38 PM (1).jpeg")


In [ ]:
system.add_person("VanjiKodi","/kaggle/working/face-recognition-dataset/Original Images/Original Images/Vanjikodi/WhatsApp Image 2025-06-13 at 6.12.38 PM (2).jpeg")

In [ ]:
system.add_person("VanjiKodi","/kaggle/working/face-recognition-dataset/Original Images/Original Images/Vanjikodi/WhatsApp Image 2025-06-13 at 6.12.38 PM.jpeg")

In [ ]:
system.add_person("VanjiKodi","/kaggle/working/face-recognition-dataset/Original Images/Original Images/Vanjikodi/WhatsApp Image 2025-06-13 at 6.12.39 PM (1).jpeg")

In [ ]:
system.add_person("VanjiKodi","/kaggle/working/face-recognition-dataset/Original Images/Original Images/Vanjikodi/WhatsApp Image 2025-06-13 at 6.12.39 PM (2).jpeg")

In [ ]:
system.add_person("VanjiKodi","/kaggle/working/face-recognition-dataset/Original Images/Original Images/Vanjikodi/WhatsApp Image 2025-06-13 at 6.12.39 PM.jpeg")

In [ ]:
system.add_person("VanjiKodi","/kaggle/working/face-recognition-dataset/Original Images/Original Images/Vanjikodi/WhatsApp Image 2025-06-13 at 6.12.40 PM (1).jpeg")

In [ ]:
system.add_person("VanjiKodi","/kaggle/working/face-recognition-dataset/Original Images/Original Images/Vanjikodi/WhatsApp Image 2025-06-13 at 6.12.40 PM (2).jpeg")

In [ ]:
system.add_person("VanjiKodi","/kaggle/working/face-recognition-dataset/Original Images/Original Images/Vanjikodi/WhatsApp Image 2025-06-13 at 6.12.40 PM.jpeg")

In [ ]:
system.add_person("VanjiKodi","/kaggle/working/face-recognition-dataset/Original Images/Original Images/Vanjikodi/WhatsApp Image 2025-06-13 at 6.12.41 PM (1).jpeg")

In [ ]:
system.add_person("VanjiKodi","/kaggle/working/face-recognition-dataset/Original Images/Original Images/Vanjikodi/WhatsApp Image 2025-06-13 at 6.12.41 PM (2).jpeg")

In [ ]:
system.add_person("VanjiKodi","/kaggle/working/face-recognition-dataset/Original Images/Original Images/Vanjikodi/WhatsApp Image 2025-06-13 at 6.12.41 PM (3).jpeg")

In [ ]:
system.add_person("VanjiKodi","/kaggle/working/face-recognition-dataset/Original Images/Original Images/Vanjikodi/WhatsApp Image 2025-06-13 at 6.12.41 PM.jpeg")

In [ ]:
system.add_person("VanjiKodi","/kaggle/working/face-recognition-dataset/Original Images/Original Images/Vanjikodi/WhatsApp Image 2025-06-13 at 6.12.42 PM.jpeg")

In [ ]:
results = system.recognize_faces_in_image("/kaggle/input/vanjitest/WhatsApp Image 2025-06-13 at 6.12.37 PM.jpeg")  # To recognize


In [ ]:
results = system.recognize_faces_in_image("/kaggle/input/attai1/WhatsApp Image 2025-06-13 at 6.50.11 PM.jpeg")  # To recognize


In [ ]:
import shutil

# Zip the entire working directory
shutil.make_archive('/kaggle/working', 'zip', '/kaggle/working')

# Then use this to download
from IPython.display import FileLink
FileLink(r'/kaggle/working/my_data_backup.zip')

In [ ]:
import shutil

# Folder to be zipped
folder_to_zip = '/kaggle/working/face-recognition-dataset/Original Images/Original Images'

# Destination zip path
zip_path = '/kaggle/working/my_data_backup1.zip'

# Create the zip file
shutil.make_archive(base_name=zip_path.replace('.zip', ''), format='zip', root_dir=folder_to_zip)

print("Zip file created!")

In [ ]:
from IPython.display import FileLink
FileLink('/kaggle/working/my_data_backup.zip')


In [ ]:
import os
print(os.listdir('/kaggle/working'))


In [ ]:
from IPython.display import FileLink
FileLink('/kaggle/working/my_data_backup1.zip')
